In [112]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Layout, Dropdown, Button, Output, Checkbox, Label
from IPython.display import display, clear_output
from __future__ import print_function
import json
from IPython.display import display
from tools import simplify_smiles
from ipywidgets import GridspecLayout

In [113]:
# all data from CSD
f = open("new_struct2torsion.json")
CCDC_torsions = json.load(f)

f = open("new_struct2angle.json")
CCDC_angles = json.load(f)

f = open("new_struct2bond.json")
CCDC_bonds = json.load(f)

In [114]:
# clean up the CSD torsion smiles
cleaned_CCDC_torsions = simplify_smiles(CCDC_torsions)
print(f"{len(cleaned_CCDC_torsions)}, {len(CCDC_torsions)}")

14642, 56316


In [115]:
# clean up the CSD angle smiles
cleaned_CCDC_angles = simplify_smiles(CCDC_angles)
print(f"{len(cleaned_CCDC_angles)}, {len(CCDC_angles)}")

7215, 18179


In [116]:
# clean up the CSD bond smiles
cleaned_CCDC_bonds = simplify_smiles(CCDC_bonds)
print(f"{len(cleaned_CCDC_bonds)}, {len(CCDC_bonds)}")

2114, 3849


In [117]:
# all data from HCSD
f = open("CCDCH_torsion.json")
HCSD_torsions = json.load(f)

f = open("CCDCH_angles.json")
HCSD_angles = json.load(f)

f = open("CCDCH_bonds.json")
HCSD_bonds = json.load(f)

In [118]:
# clean up the H CSD torsion smiles
cleaned_HCSD_torsions = simplify_smiles(HCSD_torsions)
print(f"{len(cleaned_HCSD_torsions)}, {len(HCSD_torsions)}")

4577, 6121


In [119]:
# clean up the H CSD angle smiles
cleaned_HCSD_angles = simplify_smiles(HCSD_angles)
print(f"{len(cleaned_HCSD_angles)}, {len(HCSD_angles)}")

2060, 2750


In [120]:
# clean up the H CSD bond smiles
cleaned_HCSD_bonds = simplify_smiles(HCSD_bonds)
print(f"{len(cleaned_HCSD_bonds)}, {len(HCSD_bonds)}")

659, 856


In [121]:
# all data from MTD
f = open("MTDtorsion.json")
MTD_torsions = json.load(f)

f = open("MTDangle.json")
MTD_angles = json.load(f)

f = open("MTDbond.json")
MTD_bonds = json.load(f)

In [122]:
# clean up the MTD torsion smiles
cleaned_MTD_torsions = simplify_smiles(MTD_torsions)
print(f"{len(cleaned_MTD_torsions)}, {len(MTD_torsions)}")

63, 237


In [123]:
# clean up the MTD angle smiles
cleaned_MTD_angles = simplify_smiles(MTD_angles)
print(f"{len(cleaned_MTD_angles)}, {len(MTD_angles)}")

37, 117


In [124]:
# clean up the MTD bond smiles
cleaned_MTD_bonds = simplify_smiles(MTD_bonds)
print(f"{len(cleaned_MTD_bonds)}, {len(MTD_bonds)}")

16, 49


Select up to 4 atoms to get internal coordinates

Use 2 atoms to get their bond length, 3 to get the angle and 4 to get the dihedral angle

In [129]:
def adjust_string_for_special_cases(s, special_cases):
    # Replace each special case with a single character (e.g., a marker)
    for case in special_cases:
        s = s.replace(case, "X")  # Assuming "X" does not appear in your string
    return s

def get_adjusted_length(s, special_cases):
    adjusted_s = adjust_string_for_special_cases(s, special_cases)
    return len(adjusted_s)

def select_dictionaries(selected_atoms):
    chosen_dict = {}
    chosen_dict_own = {}

    multi_letter_atoms = ['Cl', 'Br']

    adjusted_length = get_adjusted_length(selected_atoms, multi_letter_atoms)

    # Check the length of selected_atoms
    if adjusted_length == 2:
    # If selected_atoms is 2 atoms, use own_bonds
        chosen_dict_own = cleaned_MTD_bonds
    elif adjusted_length == 3:
    # If selected_atoms is 3 atoms, use own_angles
        chosen_dict_own = cleaned_MTD_angles
    elif adjusted_length == 4:
    # If selected_atoms is 4 atoms, use own_torsions
        chosen_dict_own = cleaned_MTD_torsions
    else:
        print("Invalid number of atoms")   
    # Check if "H" is in selected_atoms
    if "H" in selected_atoms:
    # If "H" is in selected_atoms, use the dictionaries with "H"
        if adjusted_length == 2:
            chosen_dict = cleaned_HCSD_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_HCSD_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_HCSD_torsions
        else:
            print("Invalid number of atoms")
    else:
    # If "H" is not in selected_atoms, use the dictionaries without "H"
        if adjusted_length == 2:
            chosen_dict = cleaned_CCDC_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_CCDC_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_CCDC_torsions
        else:
            print("Invalid number of atoms")
            

    return chosen_dict, chosen_dict_own

def draw_plot(selected_atoms, chosen_dict, chosen_dict_own ,fig, index):
    # check the selected_atoms and its reverse in the dictionary
    multi_letter_atoms = ['Cl', 'Br']
    selected_atoms_list = []



    # Function to reverse the order of atoms while keeping multi-letter atoms intact
    def reverse_atoms(atoms):
        result = []
        i = 0
        while i < len(atoms):
            # Check if the current and next character form a multi-letter atom
            if i + 1 < len(atoms) and atoms[i:i+2] in multi_letter_atoms:
                result.insert(0, atoms[i:i+2])
                i += 2
            else:
                result.insert(0, atoms[i])
                i += 1
        return ''.join(result)

                # Parse selected_atoms to handle multi-letter atoms correctly
    i = 0
    while i < len(selected_atoms):
        if i + 1 < len(selected_atoms) and selected_atoms[i:i+2] in multi_letter_atoms:
            selected_atoms_list.append(selected_atoms[i:i+2])
            i += 2
        else:
            selected_atoms_list.append(selected_atoms[i])
            i += 1
    
    if selected_atoms not in chosen_dict_own and selected_atoms not in chosen_dict:
        selected_atoms = reverse_atoms(selected_atoms)
      
    # rename lists for better differentiation
    CCDC_data = chosen_dict.get(selected_atoms)
    xyz_data = chosen_dict_own.get(selected_atoms)
    
    if CCDC_data is None or xyz_data is None:
        print(f"{selected_atoms} not found in database!")
        return

    print("Internal Coordinates:", selected_atoms)

    # define counts and bins for CCDC and own data
    counts, bins_CCDC_data = np.histogram(CCDC_data, bins=100, range=(min(CCDC_data*1.1), max(CCDC_data*1.1)))
    counts_CCDC_data = counts / counts.max()

    counts, bins_xyz_data = np.histogram(xyz_data, bins=bins_CCDC_data, range=(min(xyz_data*1.1), max(xyz_data*1.1)))
    counts_xyz_data = counts / counts.max()

    # define your labels
    label_atoms = '-'.join(selected_atoms_list)
    label_bonds = f"{label_atoms} Bond Length / Å "
    label_angles = f"{label_atoms} Angle / $\degree$"
    label_dihedrals = f"{label_atoms} Dihedral /$\degree$"
    label_ybonds = r"$n_{bond}$ / $n_{max}$"
    label_yangles = r"$n_{angle}$ / $n_{max}$"
    label_ytorsion = r"$n_{dihedral}$ / $n_{max}$"

    # choose the label based on the chosen dictionary
    if chosen_dict_own is cleaned_MTD_bonds:
        chosen_label_x = label_bonds.format(selected_atoms=selected_atoms)
        chosen_label_y = label_ybonds
    elif chosen_dict_own is cleaned_MTD_angles:
        chosen_label_x = label_angles.format(selected_atoms=selected_atoms)
        chosen_label_y = label_yangles
    elif chosen_dict_own is cleaned_MTD_torsions:
        chosen_label_x = label_dihedrals.format(selected_atoms=selected_atoms)
        chosen_label_y = label_ytorsion
    else:
        print("Invalid dictionary")

    
    # get bar plot normalized to 1
    
    axs = fig.add_subplot(2,2,index)
    axs.bar(bins_CCDC_data[:-1], counts_CCDC_data, width=bins_CCDC_data[1]-bins_CCDC_data[0],color= "skyblue", edgecolor = "black", label="CSD Data")
    axs.bar(bins_xyz_data[:-1], counts_xyz_data, width=bins_xyz_data[1]-bins_xyz_data[0], color= "pink", edgecolor = "black", label="MTD Data", alpha=0.5)
    axs.legend(loc="upper right")
    axs.set_xlabel(chosen_label_x)
    axs.set_ylabel(chosen_label_y)
    axs.set_ylim(0,1.1)

In [130]:
atoms = ["-", "C", "H", "N", "S", "O", "P", "F", "Cl", "Br", "I"]


def on_clear_click(b):
    for i in range(4):
        if grid[i,grid.n_columns-1] is b:
            for j in range(1, 5):
                grid[i, j].value = "-"   
                
def on_button_click(b):
    
    b.description = "Update"
    
    coordinates1 = [grid[0,x+1].value for x in range(grid.n_columns-2)]
    coordinates2 = [grid[1,x+1].value for x in range(grid.n_columns-2)]
    coordinates3 = [grid[2,x+1].value for x in range(grid.n_columns-2)]
    coordinates4 = [grid[3,x+1].value for x in range(grid.n_columns-2)]


    coordinates1 = ["" if value == "-" else value for value in coordinates1]
    selected_atoms1 = "".join(coordinates1)
    coordinates2 = ["" if value == "-" else value for value in coordinates2]
    selected_atoms2 = "".join(coordinates2)
    coordinates3 = ["" if value == "-" else value for value in coordinates3]
    selected_atoms3 = "".join(coordinates3)
    coordinates4 = ["" if value == "-" else value for value in coordinates4]
    selected_atoms4 = "".join(coordinates4)
    
    with output:
        clear_output()
        fig = plt.figure()
        fig.tight_layout()
        fig.set_figheight(10)
        fig.set_figwidth(15)
        if selected_atoms1 != "":
            chosen_dict1, chosen_dict_own1 = select_dictionaries(selected_atoms1)
            draw_plot(selected_atoms1, chosen_dict1, chosen_dict_own1 ,fig, 1)
        if selected_atoms2 != "":
            chosen_dict2, chosen_dict_own2 = select_dictionaries(selected_atoms2)
            draw_plot(selected_atoms2, chosen_dict2, chosen_dict_own2, fig, 2)
        if selected_atoms3 != "":
            chosen_dict3, chosen_dict_own3 = select_dictionaries(selected_atoms3)
            draw_plot(selected_atoms3, chosen_dict3, chosen_dict_own3, fig, 3)
        if selected_atoms4 != "":
            chosen_dict4, chosen_dict_own4 = select_dictionaries(selected_atoms4)
            draw_plot(selected_atoms4, chosen_dict4, chosen_dict_own4, fig, 4)
        plt.show()

    if check.value:
        # Collect non-empty selected atoms strings
        selected_atoms = [selected_atoms1, selected_atoms2, selected_atoms3, selected_atoms4]
        internal_coordinates = [atom for atom in selected_atoms if atom != ""]

        # Use these strings to create a filename
        safe_title = "_".join(internal_coordinates)  # create a safe title with only the internal coordinates
        fig.savefig(f"{safe_title}.pdf", format="pdf", transparent=True, bbox_inches='tight', pad_inches=0)  # save the plot with the title as the filename

grid = GridspecLayout(4, 6, layout=Layout(width="700px"))

for i in range(4):
    for j in range(6):
        if j == 0:
            grid[i, j] = Label(f"Atoms for Plot {i+1}")
        elif j == 5:
            clear_button = Button(description="Clear", layout=Layout(width="100px"))
            clear_button.on_click(on_clear_click)
            grid[i, j] = clear_button
        else:
            grid[i, j] = Dropdown(options=atoms, layout=Layout(width="100px"))
    

display(grid)

check = Checkbox(value=False, description="Save Plots") # checkbox to save the plot
display(check)

output = Output()
display(output)
     
button = Button(description='Start')
button.on_click(on_button_click)
display(button)

GridspecLayout(children=(Label(value='Atoms for Plot 1', layout=Layout(grid_area='widget001')), Dropdown(layou…

Checkbox(value=False, description='Save Plots')

Output()

Button(description='Start', style=ButtonStyle())